In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")


In [ ]:
from langchain_groq import ChatGroq

llm=ChatGroq(model="qwen/qwen3-32b")
response = llm.invoke("Hello! How are you today?")
print(response.content)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

embd=(OllamaEmbeddings(model="mxbai-embed-large:335m"))

urls=[
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/"

]

docs= [WebBaseLoader(url).load() for url in urls]

# docs_list = []
# for url in urls:
#     loaded_docs = WebBaseLoader(url).load()
#     docs_list.extend(loaded_docs)

docs_list=[items for sublist in docs for items in sublist]

# docs_list = []
# for sublist in docs:
#     for item in sublist:
#         docs_list.append(item)

text_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500,chunk_overlap=0
)
doc_splits=text_splitter.split_documents(docs_list)

#Add Vector Store
vectorstore=FAISS.from_documents(
    documents=doc_splits,
    embedding=embd
)


retriever=vectorstore.as_retriever()



In [ ]:


print("Vector store created successfully!")
print(f"Number of original documents: {len(docs_list)}")
print(f"Number of document chunks (vectors): {len(doc_splits)}")

Vector store created successfully!
Number of original documents: 3
Number of document chunks (vectors): 88


In [ ]:
### Retrieval Grader

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from pydantic import BaseModel, Field


# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

# LLM with function call
llm = ChatGroq(model="qwen/qwen3-32b")
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system ="""You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keywords or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    
    Here are some examples:
    
    Example 1:
    Question: What are the main components of an LLM-powered autonomous agent?
    Document: The architecture consists of several key components: a central controller (often an LLM), long-term and short-term memory stores, planning modules, and a set of tools that the agent can use.
    Answer: yes
    
    Example 2:
    Question: What was the score of the cricket match yesterday?
    Document: The sky is blue due to a phenomenon called Rayleigh scattering, where shorter wavelengths of light are scattered more effectively by the molecules in the atmosphere.
    Answer: no
    """
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)
##chain the prompt with the LLM
retrieval_grader = grade_prompt | structured_llm_grader
question = "What is a Agent"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='yes'


In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

prompt=hub.pull("rlm/rag-prompt")

llm=ChatGroq(model="qwen/qwen3-32b")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain= prompt | llm | StrOutputParser()

generation = rag_chain.invoke({"context":docs, "question":question})
print(generation)

<think>
Okay, the user is asking "What is an Agent?" and I need to use the provided context to answer. Let me look through the documents.

The first document mentions that an agent is built with LLM as the core controller. It's part of an autonomous system. The components include Planning, Memory, and Tool Use. Planning involves breaking down tasks into subgoals and self-reflection. Memory has short-term and long-term, with long-term using external storage. Tool use allows calling APIs for additional info.

Another document repeats similar points, emphasizing LLM as the brain. The agent can handle complex tasks by decomposing them, learn from mistakes, and use external tools. Examples like AutoGPT are given.

So, putting it together: An agent here is an autonomous system using LLM as its core. It plans tasks, uses memory (both short and long-term), and interacts with tools. The key functions are decomposition, reflection, memory retention, and API calls. Need to mention it's a problem-

In [ ]:
###question rewritter 

system="""You are a question re-writter that converts the inputed question to a better version that is optimized \n
           for web and google searches . By looking at the input try to reason about the underlying intent / meaning of the question ."""

re_write_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system),
        (
            "human",
            "Here is the intitial question :\n\n {question}\n Formulate an improved question.",
        ),
    ]
)

question_rewiter=re_write_prompt |llm | StrOutputParser()
question_rewiter.invoke({"question":question})

'<think>\nOkay, the user asked, "What is a Agent." Let me break this down. First, the question is a bit vague. The word "Agent" can mean different things depending on the context. So, the main issue here is that the question lacks specificity.\n\nI need to consider possible meanings of "Agent." It could refer to a software agent in computing, a human agent in various industries like insurance or real estate, or even a character in a story. Without context, it\'s hard to know which one the user is asking about. \n\nThe user might not realize the term "Agent" is ambiguous. Their underlying intent is likely to understand the definition and different types of agents. They might be a student, a professional in a different field, or someone encountering the term for the first time. \n\nTo improve the question, I should ask them to clarify the context. Maybe rephrase the question to ask for definitions in different contexts or specify which type of agent they\'re interested in. That way, the 

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
web_search_= TavilySearchResults(k=3)
from typing import List
from typing_extensions import TypedDict

class Graph_state(TypedDict):
    """
    Represents the state of our graph.

    Attributes :
        question:question
        generation:LLM generation
        web_search: wether to add search 
        documnets:list of documents
    """
    question:str
    generation:str
    web_search:str
    documents:List[str]

In [ ]:
from langchain.schema import Document


def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}


def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question, "web_search": web_search}


def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewiter.invoke({"question": question})
    return {"documents": documents, "question": better_question}


def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]

    # Web search
    docs = web_search_.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    documents.append(web_results)

    return {"documents": documents, "question": question}


### Edges


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    web_search = state["web_search"]
    state["documents"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

In [ ]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(Graph_state)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generate
workflow.add_node("transform_query", transform_query)  # transform_query
workflow.add_node("web_search_node", web_search)  # web search

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "web_search_node")
workflow.add_edge("web_search_node", "generate")
workflow.add_edge("generate", END)

# Compile
app = workflow.compile()

In [ ]:
from IPython.display import Image, display
display(Image(app.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
app.invoke({"question":"What are the types of agent memory?"})

---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
